## Heat maps

This is a short tutorial on how to plot a 3D PES, 2D contour plot, and density plot from MD data

## Get data from MD

irstly, let's get some data from an MD run with ASE

In [1]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase import units
from asap3 import EMT
# Set up a crystal

In [2]:
size = 10
atoms = FaceCenteredCubic(directions=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
                          symbol="Cu",
                          size=(size, size, size),
                          pbc=True)

Describe the interatomic interactions with the Effective Medium Theory

In [3]:
atoms.calc = EMT()

Set the momenta corresponding to T=300K

In [4]:
MaxwellBoltzmannDistribution(atoms, temperature_K=300)

We want to run MD with constant energy using the VelocityVerlet algorithm.

In [5]:
dyn = VelocityVerlet(atoms, 5 * units.fs, trajectory='dyn.traj', logfile='md.log')  # 5 fs time step.

In [6]:
def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

Now run the dynamics

In [7]:
dyn.attach(printenergy, interval=10)
printenergy()
dyn.run(200)

Energy per atom: Epot = -0.001eV  Ekin = 0.039eV (T=300K)  Etot = 0.038eV
Energy per atom: Epot = -0.001eV  Ekin = 0.039eV (T=300K)  Etot = 0.038eV
Energy per atom: Epot = 0.029eV  Ekin = 0.009eV (T= 70K)  Etot = 0.038eV
Energy per atom: Epot = 0.020eV  Ekin = 0.019eV (T=146K)  Etot = 0.038eV
Energy per atom: Epot = 0.018eV  Ekin = 0.021eV (T=159K)  Etot = 0.038eV
Energy per atom: Epot = 0.017eV  Ekin = 0.021eV (T=163K)  Etot = 0.038eV
Energy per atom: Epot = 0.020eV  Ekin = 0.018eV (T=142K)  Etot = 0.038eV
Energy per atom: Epot = 0.020eV  Ekin = 0.019eV (T=144K)  Etot = 0.038eV
Energy per atom: Epot = 0.018eV  Ekin = 0.020eV (T=157K)  Etot = 0.038eV
Energy per atom: Epot = 0.018eV  Ekin = 0.020eV (T=154K)  Etot = 0.038eV
Energy per atom: Epot = 0.020eV  Ekin = 0.018eV (T=143K)  Etot = 0.038eV
Energy per atom: Epot = 0.019eV  Ekin = 0.020eV (T=152K)  Etot = 0.038eV
Energy per atom: Epot = 0.018eV  Ekin = 0.020eV (T=154K)  Etot = 0.038eV
Energy per atom: Epot = 0.019eV  Ekin = 0.019eV (

True

## Post processing

Now that we have generated some MD data let's analyse some Cu-Cu distances

Read in MD trajectory

In [8]:
from ase.io import Trajectory
traj = Trajectory('dyn.traj')

get Cu-Cu distances for atoms [0] to [1],[2] from the MD traj and append to list

In [9]:
bond1 = []
for atoms in traj[0:200]:
    dist1 = atoms.get_distance(0,1)
    bond1.append(dist1)

In [10]:
bond2 = []
for atoms in traj[0:200]:
    dist2 = atoms.get_distance(0,2)
    bond2.append(dist2)

## Prepare data for plotting

Here we want to open the md logfile as a pandas dataframe

In [11]:
import pandas as pd
data = pd.read_csv("md.log", sep='\s{2,}',header=None, nrows=200, skiprows=[0])
data = pd.DataFrame(data)

/var/folders/06/kkylt54j0yj__mz8g6j31tqh0000gn/T/ipykernel_55151/1563724761.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("md.log", sep='\s{2,}',header=None, nrows=200, skiprows=[0])


create variables with total energy and Cu-Cu distance

In [12]:
z = data[1].tolist()
x = bond1
y = bond2

create a dataframe for plotting 3D data

In [13]:
df = pd.DataFrame(list(zip(x,y,z)), columns=list('XYZ'))
print(df)

            X         Y       Z
0    2.552655  2.552655  152.68
1    2.562335  2.569685  152.72
2    2.571229  2.586233  152.83
3    2.578571  2.601736  153.00
4    2.583668  2.615678  153.20
..        ...       ...     ...
195  2.627055  2.592296  153.24
196  2.633113  2.589790  153.25
197  2.636289  2.583862  153.25
198  2.636739  2.575081  153.25
199  2.634710  2.564161  153.26

[200 rows x 3 columns]


## 3D PES

Let's use matplotlib to plt a 3D PES of our data

In [14]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.plasma, linewidth=0.01)
fig.colorbar( surf, shrink=0.5, aspect=5)
ax.view_init(30, 45)
plt.ylabel('Cu0-Cu1/ Å')
plt.xlabel('Cu0-Cu2/ Å')

TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'

<Figure size 640x480 with 0 Axes>

## 2D PES

Now let's try visualising our PES as a 2D contour plot

In [ ]:
plt.tricontourf(df["X"], df["Y"], df["Z"],levels=10, cmap='plasma')
plt.colorbar()
plt.ylabel('Cu0-Cu1/ Å')
plt.xlabel('Cu0-Cu2/ Å')

## Plot density of scatter points

plot density of scatter points for Cu-Cu distances

In [ ]:
import numpy as np
import matplotlib.cm as cm
from scipy.ndimage.filters import gaussian_filter

In [ ]:
def myplot(x, y, s, bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent
fig, axs = plt.subplots(1, 2, constrained_layout=True)
sigmas = [0, 64]
for ax, s in zip(axs.flatten(), sigmas):
    if s == 0:
        ax.plot(df['X'], df['Y'], 'k.', markersize=5)
        ax.set_title("Scatter plot")
    else:
        img, extent = myplot(df['X'], df['Y'], s)
        ax.imshow(img, aspect="auto", extent=extent, origin='lower', cmap=cm.jet)
        ax.set_title("Density of scatter points")
plt.ylabel('Cu0-Cu1/ Å')
plt.xlabel('Cu0-Cu2/ Å')
plt.show()